# Lecture9　パターン認識2(クラス分類)
<div dir='rtl'>
2024.5岩政
</div>

# クラス分類（続き）

## kNN(k近傍法)
k近傍法（k-nearest neighbors algorithm）  
1.6. Nearest Neighbors http://scikit-learn.org/stable/modules/neighbors.html<br>
sklearn.neighbors.KNeighborsClassifier http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html  
距離の種類 http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import cm # maplotlibのカラーマップを使用

from sklearn.neighbors import KNeighborsClassifier


データを用意します

In [ ]:
X = np.array([ [-3.0, -2.0], [-2.0, -1.0], [-1.0, -1.0], [1.0, 1.0], [2.0, 1.0], [3.0, 2.0]])
y = np.array([0, 0, 1, 1, 1, 1])

In [ ]:
plt.scatter(X[:,0], X[:,1], c=y, cmap=cm.bwr)


近傍=3で計算をします（fit()関数)。

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X, y)

テスト用の、データ点２を２つ用意し、予測をします。

In [ ]:
X_test = np.array([ [-1.5, -1.0], [0.0, 0.5]])
print('Estimated class: ', neigh.predict(X_test))

テスト用データに対する３点と、距離を表示します。

In [ ]:
distances, indices = neigh.kneighbors(X_test)
print('Nearest index \n',indices)
print('Distance \n',distances)

予測値を重ねてみます。

In [ ]:
plt.scatter(X[:,0], X[:,1], c=y, cmap=cm.bwr)
plt.scatter(X_test[:,0], X_test[:,1],c=neigh.predict(X_test),cmap=cm.bwr,  marker="*",s=300,alpha=0.2,linewidths=2, edgecolors="orange")

### kNN irisデータの分類

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import cm

from sklearn.neighbors import KNeighborsClassifier
from sklearn import datasets
from sklearn.metrics import  accuracy_score, classification_report
from sklearn.model_selection import train_test_split


#### Xは2次元
iris変量の内，sepal（がく）と petal（花びら）の長さ[cm]を見るため
0番目と2番目のデータをXに格納

In [ ]:
# Load iris data, 150 sample
iris = datasets.load_iris()
X = iris.data[:, [0, 2]]
y = iris.target

# Split into training and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=2/15, random_state=123)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=10/15,)

fig = plt.subplots(figsize=(8,4))
plt.xlim(4.0, 8.0)
plt.ylim(0.5, 7.5)
plt.scatter(X_train[:,0], X_train[:,1], c=y_train, cmap=cm.bwr, edgecolors='k')
plt.title('Training, Blue:0, White:1, Red:2')
plt.grid()


近傍点の数kを3,7,9と変えてみて試します。

In [ ]:
k_range=(3, 7, 9)
for k in k_range:

    neigh = KNeighborsClassifier(n_neighbors=k)
    neigh.fit(X_train, y_train)

    y_test_pred = neigh.predict(X_test)
    acc = accuracy_score(y_test, y_test_pred)
    print('Accuracy =',acc)
    print(classification_report(y_test, y_test_pred))

    fig = plt.subplots(figsize=(8,4))
    plt.xlim(4.0, 8.0)
    plt.ylim(0.5, 7.5)
    plt.scatter(X_test[:,0], X_test[:,1], c=y_test_pred, cmap=cm.bwr, edgecolors='k')
    plt.grid()
    plt.title('Predicttion, Blue:0, White:1, Red:2; with k='+str(k))


どのデータが誤っているかを調べる。

In [ ]:
print('Real       =',y_test)
print('Prediction =',y_test_pred)

In [ ]:
#上のグラフと比較してｋNNの性能を推測する。
print('  0:',X_test[0],'\n  5:',X_test[5],'\n 10:',X_test[10])

誤分類された点を表示します。

In [ ]:
miss=X_test[[0,5,10]]
missy=y_test_pred[[0,5,10]]

plt.scatter(X_test[:,0], X_test[:,1], c=y_test_pred, cmap=cm.bwr, edgecolors='k')
plt.scatter(miss[:,0], miss[:,1],c=missy,cmap=cm.bwr,  marker="*",s=300,alpha=0.2,linewidths=2, edgecolors="orange")

#### 説明変数の特徴量を4次元（全て）にして，kNNを実施
ただし，splitする際にrandom_stateを省いているため，試行ごとにトレーニングデータとテストデータの性質が異なる。

In [ ]:
# random_state be not set. random division
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=2/15)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train)

In [ ]:
_test_predt = neigh.predict(X_test)
acc = accuracy_score(y_test, _test_predt)
print('Accuracy =',acc)
print('Real       =',y_test)
print('Prediction =',_test_predt)

結果を可視化してみます。最初に誤判定した部分を抽出します。

In [ ]:
miss_list = np.where(y_test != _test_predt)[0]
miss_list

結果を表示します。

In [ ]:
miss=X_test[miss_list]
missy=y_test_pred[miss_list]

fig = plt.subplots(figsize=(8,4))
plt.xlim(4.0, 8.0)
plt.ylim(0.5, 7.5)
plt.scatter(X_test[:,0], X_test[:,1], c=y_test, cmap=cm.bwr, edgecolors='k')
plt.scatter(miss[:,0], miss[:,1],c=missy,cmap=cm.bwr,  marker="*",s=300,alpha=0.2,linewidths=2, edgecolors="orange")
plt.title('y_test, Blue:0, White:1, Red:2')
plt.grid()

## 決定木・ランダムフォレスト

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

人工的なデータをmake_blobsで作ります。

In [ ]:
from sklearn.datasets import make_blobs

X, y = make_blobs(n_samples=300, centers=4,
                  random_state=0, cluster_std=1.0)
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='rainbow');

In [ ]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier().fit(X, y)

In [ ]:
def visualize_classifier(model, X, y, ax=None, cmap='rainbow'):
    ax = ax or plt.gca()

    # Plot the training points
    ax.scatter(X[:, 0], X[:, 1], c=y, s=30, cmap=cmap,
               clim=(y.min(), y.max()), zorder=3)
    ax.axis('tight')
    ax.axis('off')
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()

    # fit the estimator
    model.fit(X, y)
    xx, yy = np.meshgrid(np.linspace(*xlim, num=200),
                         np.linspace(*ylim, num=200))
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

    # Create a color plot with the results
    n_classes = len(np.unique(y))
    contours = ax.contourf(xx, yy, Z, alpha=0.3,
                           levels=np.arange(n_classes + 1) - 0.5,
                           cmap=cmap, clim=(y.min(), y.max()),
                           zorder=1)

    ax.set(xlim=xlim, ylim=ylim)

In [ ]:

visualize_classifier(DecisionTreeClassifier(max_depth=8), X, y)


決定木を可視化してみます。

In [ ]:
from sklearn import tree
clf=DecisionTreeClassifier(max_depth=8).fit(X, y)
tree.plot_tree(clf)
#plt.savefig("lec8_decision_tree_plot.png",  dpi=300)

### 注意
分類の様子を可視化します、manabaで配布した、helpers_05_08.py をgoogle colab環境にアップロードして使います。

In [ ]:
from helpers_05_08 import visualize_tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_blobs


fig, ax = plt.subplots(1, 2, figsize=(16, 6))
fig.subplots_adjust(left=0.02, right=0.98, wspace=0.1)

X, y = make_blobs(n_samples=300, centers=4,
                  random_state=0, cluster_std=1.0)

for axi, depth in zip(ax, range(1, 3)):
    model = DecisionTreeClassifier(max_depth=depth)
    visualize_tree(model, X, y, ax=axi)
    axi.set_title('depth = {0}'.format(depth))

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16, 6))
fig.subplots_adjust(left=0.02, right=0.98, wspace=0.1)

for axi, depth in zip(ax, range(3, 5)):
    model = DecisionTreeClassifier(max_depth=depth)
    visualize_tree(model, X, y, ax=axi)
    axi.set_title('depth = {0}'.format(depth))

In [ ]:
# helpers_05_08 is found in the online appendix
import helpers_05_08
helpers_05_08.plot_tree_interactive(X, y);

In [ ]:
# helpers_05_08 is found in the online appendix
import helpers_05_08
helpers_05_08.randomized_tree_interactive(X, y);

### ランダムフォレスト

単純にたくさん単独の決定木をあつめて予測するには、BaggingClassifierを使います。

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

tree = DecisionTreeClassifier()
bag = BaggingClassifier(tree, n_estimators=100, max_samples=0.8,
                        random_state=1)

bag.fit(X, y)
visualize_classifier(bag, X, y)

それぞれの木を作るとき、ランダムにデータを間引くなどをして、過適合を避けながら、木の集合（森）をつくのがRandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, random_state=0)
visualize_classifier(model, X, y);

#### Example: Random Forest for Classifying Digits

手書き文字の画像分類にランダムフォレストを使う例紹介します。

In [ ]:
from sklearn.datasets import load_digits
digits = load_digits()
digits.keys()

In [ ]:
# set up the figure
fig = plt.figure(figsize=(6, 6))  # figure size in inches
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)

# plot the digits: each image is 8x8 pixels
for i in range(64):
    ax = fig.add_subplot(8, 8, i + 1, xticks=[], yticks=[])
    ax.imshow(digits.images[i], cmap=plt.cm.binary, interpolation='nearest')

    # label the image with the target value
    ax.text(0, 7, str(digits.target[i]))

In [ ]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(digits.data, digits.target,
                                                random_state=0)
model = RandomForestClassifier(n_estimators=1000)
model.fit(Xtrain, ytrain)
ypred = model.predict(Xtest)

予測結果(ypred)は数値になります。

In [ ]:
ypred

In [ ]:
from sklearn import metrics
print(metrics.classification_report(ypred, ytest))

予測値がどれだけ正解と当たったかはずれたか、外れたら、何になったかをconfusion_matrixにより計算します。

結構あたっていることがわかります。

In [ ]:
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(ytest, ypred)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False)
plt.xlabel('true label')
plt.ylabel('predicted label');

## ナイーブベイズ(Naive Bays)

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

### Gaussian Naive Bayes

In [ ]:
from sklearn.datasets import make_blobs
X, y = make_blobs(100, 2, centers=2, random_state=2, cluster_std=1.5)
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='RdBu');

In [ ]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X, y);

In [ ]:
rng = np.random.RandomState(0)
Xnew = [-6, -14] + [14, 18] * rng.rand(2000, 2)
ynew = model.predict(Xnew)

In [ ]:
ynew

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='RdBu')
lim = plt.axis()
plt.scatter(Xnew[:, 0], Xnew[:, 1], c=ynew, s=20, cmap='RdBu', alpha=0.1)
plt.axis(lim);

In [ ]:
from sklearn.datasets import make_blobs

X, y = make_blobs(n_samples=300, centers=4,
                  random_state=0, cluster_std=1.0)
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='rainbow');

In [ ]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X, y);

In [ ]:
rng = np.random.RandomState(0)
Xnew = [-4, -2] + [8, 12] * rng.rand(2000, 2)
ynew = model.predict(Xnew)

In [ ]:
#plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='RdBu')
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='rainbow');
lim = plt.axis()
plt.scatter(Xnew[:, 0], Xnew[:, 1], c=ynew, s=20, cmap='rainbow', alpha=0.1)
plt.axis(lim);